In [ ]:
pip install nums_from_string

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import datetime
import nums_from_string
import csv
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
data = []
cities=['Mumbai', 'Gurgaon','Noida','Ghaziabad','Greater-Noida','Bangalore','Pune','Hyderabad','Kolkata','Chennai',
        'New-Delhi','Ahmedabad','Navi-Mumbai','Thane','Faridabad','Bhubaneswar','Bokaro-Steel-City','Vijayawada','Vrindavan', 'Bhopal',
        'Gorakhpur','Jamshedpur','Agra','Allahabad','Jodhpur''Aurangabad','Jaipur','Mangalore','Nagpur','Guntur','Navsari','Palghar','Salem','Haridwar','Durgapur',
        'Madurai','Manipal','Patna','Ranchi','Raipur','Sonipat','Kottayam','Kozhikode','Thrissur','Tirupati','Trivandrum','Trichy','Udaipur','Vapi','Varanasi',
        'Vadodara','Visakhapatnam','Surat','Kanpur','Kochi','Mysore','Goa','Bhiwadi','Lucknow','Nashik','Guwahati','Chandigarh','Indore','Coimbatore','Dehradun']

In [ ]:
today = datetime.datetime.today().strftime ('%Y-%m-%d')
def get_date_posted(ago_count):
  Previous_Date = datetime.datetime.today() - datetime.timedelta(days=ago_count)
  previous_d_for = Previous_Date.strftime ('%d/%m/%Y')
  return previous_d_for

def scrape(city):
  for i in range(1,20):
    
    url = "https://www.magicbricks.com/property-for-rent/commercial-real-estate?proptype=Commercial-Office-Space,Office-ITPark-SEZ,Commercial-Shop,Commercial-Showroom&page="+str(i)+"&cityName="+city
    response = requests.get(url)
    response = response.content
    soup = BeautifulSoup(response, 'html.parser')
    cards = soup.find_all('div', class_='mb-srp__card')

    prop_age = None
    water = None
    carpet_area = None
    ready_state = None
    furnishing = None
    washroom = None
    parking = None
    pantry = None
    overlooking = None
    facing = None
    locality = None
    per_sqft = None
    desc = None
    date_posted = None

    for card in cards:
      try:
        posted_date = card.find(class_="mb-srp__card__photo__fig--post").text
        if 'today' in posted_date or 'Today' in posted_date:
          date_posted = get_date_posted(0)
        elif 'ago' in posted_date:
          ago_date_count = int(nums_from_string.get_nums(posted_date)[0])
          if 'days' in posted_date:
              date_posted = get_date_posted(ago_date_count)
          elif 'weeks' in posted_date:
              date_posted = get_date_posted((ago_date_count*7))
          else:
              date_posted = get_date_posted((ago_date_count*30))
        elif 'yesterday' in posted_date or 'Yesterday' in posted_date:
            date_posted = get_date_posted(1)
        else:
            date_posted = posted_date.replace('Posted: ','')
      except:
          posted_date = None

      try: 
          link = card.find('a', class_='mb-srp__card__society--name')['href']
      except:
          link = None
      try: 
          retailer = card.find('div', class_='mb-srp__card__ads__info--name').text
      except:
          retailer = None
      bhks = card.find('h2', class_='mb-srp__card--title').text
      prop_locality = bhks.split(', ')
      bhk = prop_locality[0][0:5]
      del prop_locality[0]
      try:
        locality = prop_locality[0]
      except:
        locality = city
      childs = card.find_all('div', class_='mb-srp__card__summary-commercial__list--item')
      description = card.find('div', class_='mb-srp__card--desc')
      try: 
          desc = description.find('p').text.replace('"', '')
      except:
          desc = None
      price = card.find('div', class_='mb-srp__card__price--amount').text.replace('₹', '')
      try: 
          per_sqft = card.find('div', class_='mb-srp__card__price--size').text.replace('₹', '')
      except:
          per_sqft = None
      for elem in childs:

        try:
          if 'Carpet Area' in elem.text:
            carpet_area = elem.text.replace('Carpet Area', '')
        except:
          carpet_area = None

        try:
          if 'Overlooking' in elem.text:
            overlooking = elem.text.replace('Overlooking', '')
        except:
          overlooking = None

        try:
          if 'Furnishing Status' in elem.text:
            furnishing = elem.text.replace('Furnishing Status', '')
        except:
          furnishing = None

        try:
          if 'Parking' in elem.text:
            parking = elem.text.replace('Parking', '')
        except:
          parking = None

        try:
          if 'Pantry' in elem.text:
            pantry = elem.text.replace('Pantry', '')
        except:
          pantry = None

        try:
          if 'Facing' in elem.text:
            facing = elem.text.replace('Facing', '')
        except:
          facing = None

        try:
          if 'Washroom' in elem.text:
            washroom = elem.text.replace('Washroom', '')
        except:
          washroom = None

        try:
          if 'Water Availability' in elem.text:
            water = elem.text.replace('Water Availability', '')
        except:
          water = None

        try:
          if 'Property Age' in elem.text:
            prop_age = elem.text.replace('Property Age', '')
        except:
          prop_age = None

        try:
            data.append([date_posted, link, retailer, carpet_area, locality, city, washroom, price, per_sqft, parking, facing, pantry, water, prop_age, ready_state, desc])
        except:
          pass
  df = pd.DataFrame(data, columns=['Date Posted', 'Link', 'Retailer', 'Carpet Area', 'Locality', 'City', 'Washroom', 'Price', 'Price/Sqft', 'Parking', 'Facing', 'Pantry', 'Water Availability', 'Property Age', 'Status', 'Description'])
  df.to_csv('Comm_{0}.csv'.format(str(city)), index=False)
  df.drop_duplicates(subset='Link', keep='first', inplace=True)
  df.insert(loc=0, column='No', value=np.arange(len(df)))
  try:
    df.drop(index=df.index[0], axis=0, inplace=True)
  except:
    df = df.fillna(0)
  df.to_csv('{0}_{1}_CommLease.csv'.format(str(city), today), index=False)

  # Function to convert a CSV to JSON
  def make_json(csvFilePath, jsonFilePath):
    
    data = {}
    
    with open(csvFilePath, encoding='utf-8') as csvf:
      csvReader = csv.DictReader(csvf)

      for rows in csvReader:

        key = rows['No']
        data[key] = rows

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
      jsonf.write(json.dumps(data, indent=4))

  csvFilePath = r'{0}_{1}_CommLease.csv'.format(str(city), today)
  jsonFilePath = r'{0}_{1}_CommLease.json'.format(str(city), today)

  make_json(csvFilePath, jsonFilePath)

In [ ]:
for city in cities:
  scrape(city)

KeyboardInterrupt: ignored